<a href="https://colab.research.google.com/github/vedu-13/Codtech-Cyber-Security-/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup

def simple_scanner(url):
    """
    A basic web scanner that makes a request to a URL.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        print(f"Successfully accessed: {url}")
        # You can process the content here using BeautifulSoup if needed
        # soup = BeautifulSoup(response.content, 'html.parser')
        # print(soup.title)

    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {e}")

# Example usage (replace with a target URL you have permission to scan)
# simple_scanner("http://example.com")

In [5]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def simple_scanner(url):
    """
    A basic web scanner that makes a request to a URL and performs basic SQL injection and XSS checks.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        print(f"Successfully accessed: {url}")

        soup = BeautifulSoup(response.content, 'html.parser')

        # Basic SQL Injection Check
        print("\n--- Performing SQL Injection Checks ---")
        input_tags = soup.find_all(['input', 'textarea'])
        for tag in input_tags:
            if tag.get('name'):
                print(f"Found input field: {tag.get('name')}")
                # Simple payloads to test for error-based SQL injection
                sql_payloads = ["'", "' OR '1'='1", "\" OR \"1\"=\"1", "') OR ('1'='1"]
                for payload in sql_payloads:
                    test_url = url
                    # Assuming GET requests for simplicity; POST requires more complex handling
                    if '?' in test_url:
                        test_url += '&' + tag.get('name') + '=' + urllib.parse.quote(payload)
                    else:
                        test_url += '?' + tag.get('name') + '=' + urllib.parse.quote(payload)

                    try:
                        test_response = requests.get(test_url)
                        # Look for common database error messages in the response content
                        if any(error in test_response.text for error in ['sql syntax', 'mysql_fetch_array()', 'ORA-', 'Microsoft JET Database Engine']):
                            print(f"Potential SQL Injection vulnerability found in field '{tag.get('name')}' with payload: {payload}")
                            print(f"Test URL: {test_url}")

                    except requests.exceptions.RequestException as e:
                        print(f"Error testing payload {payload} on {test_url}: {e}")

        # Basic XSS Check
        print("\n--- Performing XSS Checks ---")
        xss_payloads = ["<script>alert('XSS')</script>", "<img src=x onerror=alert('XSS')>"]
        for tag in input_tags:
             if tag.get('name'):
                print(f"Found input field for XSS check: {tag.get('name')}")
                for payload in xss_payloads:
                    test_url = url
                    if '?' in test_url:
                        test_url += '&' + tag.get('name') + '=' + urllib.parse.quote(payload)
                    else:
                        test_url += '?' + tag.get('name') + '=' + urllib.parse.quote(payload)

                    try:
                        test_response = requests.get(test_url)
                        # Check if the payload is reflected in the response
                        if payload in test_response.text:
                             print(f"Potential XSS vulnerability found in field '{tag.get('name')}' with payload: {payload}")
                             print(f"Test URL: {test_url}")

                    except requests.exceptions.RequestException as e:
                         print(f"Error testing payload {payload} on {test_url}: {e}")



    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {e}")

# Example usage (replace with a target URL you have permission to scan)
# simple_scanner("http://example.com")